In [9]:
import yahoo_fin.stock_info as si
import pandas as pd
import math as math

In [13]:
#Nasdaq: tickers_nasdaq()
#S&P500: tickers_sp500()
#Others: tickers_other()
nasdaq = si.tickers_nasdaq()

In [60]:
import requests
import json
def callApi(url, stock):

    querystring = {"symbol": stock }

    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "b057006049msh2dd6f48e0283787p151e0djsnf76cd322583c"
        }
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [61]:
def priceToBook(stock):
    result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary", stock)
    if "raw" in result["defaultKeyStatistics"]["priceToBook"].keys():
        return  result["defaultKeyStatistics"]["priceToBook"]["raw"]
    else:
        return 0.0

In [3]:
sp500 =['LUV']#['LUV','TELA', 'FB', 'MAR', 'JPM','HTZ', 'DAL' ]
dow_stats = {}
data = {} 
for ticker in sp500:
    data = si.get_stats_valuation(ticker)
    data = data.iloc[:,:2]
    data.columns = ["Attribute", "Recent < 15"]
    data = data[data["Attribute"]=="Trailing P/E"]
    if math.isnan(float(data["Recent < 15"])): 
        data = data.iloc[:,:3]
        data = data[data["Attribute"]=="Forward P/E"]
        print(data)
    trailing_PE = float(data["Recent < 15"])
    price_Book = priceToBook(ticker)
    data["Ticker"] = ticker
    data["price_Book < 1.5"] = price_Book
    validate = trailing_PE * price_Book
    data["validate < 22.5"] = validate
    if validate < 22.5:
        data["Need a look"] = True
    else: 
        data["Need a look"] = False

NameError: name 'priceToBook' is not defined

In [67]:
def peValue(stock):
    result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary", stock)
    trailingPE = {}
    forwardPE = {}
    if not("summaryDetail" in result.keys()):
        return {"trailingPE": 0.0, "forwardPE":0.0}  
    if ("trailingPE" in result["summaryDetail"].keys()) and ("raw" in result["summaryDetail"]["trailingPE"].keys()):
        trailingPE =  result["summaryDetail"]["trailingPE"]["raw"]
    else: 
        trailingPE = 0.0
    if "forwardPE" in result["summaryDetail"].keys() and "raw" in result["summaryDetail"]["forwardPE"].keys():
        forwardPE =   result["summaryDetail"]["forwardPE"]["raw"]
    else:
        forwardPE = 0.0
    return {"trailingPE": trailingPE, "forwardPE":forwardPE} 

In [29]:
peValue('COUP')

{'trailingPE': 0.0, 'forwardPE': 525.75806}

In [68]:

tickers = ['LUV','TELA', 'FB', 'MAR', 'JPM','HTZ', 'DAL' ]
dataSet = [] 
for ticker in tickers:
    price_Book = priceToBook(ticker)
    pe_Value = peValue(ticker)
    data["Ticker"] = ticker
    data["Price_Book < 1.5"] = price_Book
    data["TrailingPE < 15"] = pe_Value["trailingPE"]
    data["ForwardPE < 15"] = pe_Value["forwardPE"]
    validate = 0.0
    
    if pe_Value['trailingPE'] != 0:
        validate = pe_Value['trailingPE'] * price_Book
        data["Trailing Value < 22.5"] = validate
        if validate < 22.5 and price_Book != 0:
            data["Trailing Look"] = True
        else: 
            data["Trailing Look"] = False 
    else:
        data["Trailing Value < 22.5"] = 0
        data["Trailing Look"] = False
        
    if pe_Value['forwardPE'] != 0 and price_Book != 0:
        validate = pe_Value['forwardPE'] * price_Book
        data["Forward Value < 22.5"] = validate
        if validate < 22.5:
            data["Forward Look"] = True
        else: 
            data["Forward Look"] = False
    else:
        data["Forward Value < 22.5"] = 0
        data["Forward Look"] = False
    dataSet.append(data.copy())


In [69]:
pd.DataFrame(dataSet)

,Ticker,Price_Book < 1.5,TrailingPE < 15,ForwardPE < 15,Trailing Value < 22.5,Trailing Look,Forward Value < 22.5,Forward Look
0,LUV,2.104550,127.245900,23.521214,267.795308,False,49.501562,False
1,TELA,3.066606,0.000000,-7.982248,0.000000,False,-24.478409,True
2,FB,7.577736,35.908535,28.932022,272.105413,False,219.239236,False
3,MAR,0.000000,75.576100,34.500000,0.000000,False,0.000000,False
4,JPM,1.336063,13.759540,11.651927,18.383618,True,15.567713,True
5,HTZ,0.421906,0.000000,-1.324561,0.000000,False,-0.558840,True
6,DAL,2.338621,0.000000,16.661459,0.000000,False,38.964840,False
